In [1]:
data_var = '2024-01-19'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11606,2024-01-19,França Pro B,16:00,Pau-Orthez,Nantes,1.58,2.29,155.5,1.88,1.79,-4.5,2.01,1.72,tzx7VMQc,0.632911,0.436681,0.531915,0.558659,0.069593,151.000,42.742749,0.283065,0.6,1.341641,2.236068,191.26,2.006,0.558596,0.278463,-23.0,159.474,47.067276,0.295141,1.8,1.643168,0.912871,222.75,2.042,0.667960,0.327110,4.0,73,75,2.62,2.97,114.024,188.098,0.259455,0.034681,0.109952,-2.10,-0.420,-1.380952,0.000000,0.0,0.000000,-0.46,-0.092,-14.021739,0.000000,0.0,0.000000
11607,2024-01-19,França Pro B,16:00,Poitiers,Denain-Voltaire,1.37,2.93,157.5,1.85,1.85,-7.5,2.01,1.72,pSYNRvQM,0.729927,0.341297,0.540541,0.540541,0.071224,202.438,14.939097,0.073796,1.8,1.643168,0.912871,220.32,2.670,0.447884,0.167747,6.0,177.144,85.180109,0.480852,1.2,1.643168,1.369306,109.52,2.432,1.337225,0.549846,-24.0,72,74,3.06,1.48,169.522,0.000,0.513064,0.000000,0.109952,2.55,0.510,0.725490,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11608,2024-01-19,França Pro B,16:30,Vichy,Boulazac,1.39,2.84,160.5,1.87,1.79,-7.5,2.09,1.66,x6oH8aJp,0.719424,0.352113,0.534759,0.558659,0.071537,135.666,36.660386,0.270225,2.4,1.341641,0.559017,130.26,1.648,0.442685,0.268620,14.0,121.452,21.581332,0.177694,2.4,1.341641,0.559017,131.92,1.460,0.177623,0.121660,28.0,78,97,1.67,1.36,105.686,0.000,0.484778,0.030912,0.162163,-0.06,-0.012,-32.500000,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11609,2024-01-19,Indonésia Ibl,08:00,Satya Wacana,Rajawali Medan,2.34,1.55,141.5,1.88,1.79,2.5,2.05,1.68,Ag2TGP1f,0.427350,0.645161,0.531915,0.558659,0.072512,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,85,0.00,1.65,0.000,0.000,0.287205,0.034681,0.140284,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11610,2024-01-19,Indonésia Ibl,10:00,Pelita Jaya,Tangerang Hawks,1.14,5.19,146.5,1.92,1.78,-14.5,2.06,1.68,lp1XFqH0,0.877193,0.192678,0.520833,0.561798,0.069871,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,76,81,1.32,1.48,0.000,0.000,0.904829,0.053511,0.143690,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11727,2024-01-19,Chipre Divisão A,14:30,Anorthosis,AEK,2.13,1.66,154.5,1.85,1.85,1.5,2.00,1.72,h0d41xPa,0.469484,0.602410,0.540541,0.540541,0.071893,177.046,122.596300,0.692455,2.4,1.341641,0.559017,164.34,2.238,1.793201,0.801252,102.0,97.288,16.313236,0.167680,3.0,0.000000,0.000000,92.04,1.180,0.226936,0.192319,83.0,83,78,1.98,1.18,0.000,0.000,0.175377,0.000000,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11728,2024-01-19,China Cba,08:35,Zhejiang Guangsha,Jiangsu Dragons,1.05,8.90,191.5,1.85,1.89,-18.5,2.03,1.68,4K1UEEza,0.952381,0.112360,0.540541,0.529101,0.064741,140.336,31.320733,0.223184,1.2,1.643168,1.369306,139.86,1.372,0.320967,0.233941,-11.0,422.558,201.261653,0.476294,0.0,0.000000,NaN,238.5

## Apostas do dia

### Back Home V3 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11632,21:00,Eua Ncaa,St. Francis (PA),LIU Sharks,1.69,Back Home
11671,13:00,Catar Liga De Basquete Do Catar,Al-Rayyan,Al Shamal,1.54,Back Home
11676,05:30,Austrália Nbl,Tasmania JackJumpers,Melbourne United,2.12,Back Home
11690,22:00,Eua Nba,New Orleans Pelicans,Phoenix Suns,1.75,Back Home
11701,08:35,China Cba,Shandong,Shenzhen,2.57,Back Home
11708,01:00,Eua Ncaa,Stanford,Washington State,1.63,Back Home
